# Assignment 1: Linguistic Analysis using NLP
*Author: Aleksander Moeslund Wael*

In [76]:
# Importing packages
import os
import spacy
import pandas as pd

In [86]:
# loading the spacy model
nlp = spacy.load("en_core_web_md")

# setting the path to the folder with the corpus
path_to_folder = os.path.join(os.getcwd(), "in", "USEcorpus")
subfolders = os.listdir(path_to_folder)

for x in subfolders: # NEED FULL PATH
    folder = os.path.join(path_to_folder, x) # NEED FULL PATH
    # create a DF with name of subfolder
    df = pd.DataFrame(columns=["text_name", "RelFreq_NOUN", "RelFreq_VERB", "RelFreq_ADJ", "RelFreq_ADV", "Unique_PER", "Unique_LOC", "Unique_ORG"])

    # load as spacy doc
    for y in os.listdir(folder): # NEED FULL PATH
        file_path = os.path.join(folder, y) # NEED FULL PATH
        f = open(file_path, "r", encoding="ISO-8859-1")
        file = f.read()
        
        # load as spacy doc
        file_text = nlp(file)
        
         # Creating empty lists for each category
        non_words_counter = 0
        nouns = []
        verbs = []
        adjectives = []
        adverbs = []
        PER = []
        LOC = []
        ORG = []

        # looping over each token in the text and appending the tokens to the correct list
        for token in file_text:
            if token.pos_ == "X" or token.pos == "PUNCT" or token.pos == "SPACE" or token.pos == "SYM":
                non_words_counter += 1
            if token.pos_ == "NOUN":
                nouns.append(token.text)
            if token.pos_ == "VERB":
                verbs.append(token.text)
            if token.pos_ == "ADJ":
                adjectives.append(token.text)
            if token.pos_ == "ADV":
                adverbs.append(token.text)
            if token.ent_type_ == "PER":
                PER.append(token.text)
            if token.ent_type_ == "LOC":
                LOC.append(token.text)
            if token.ent_type_ == "ORG":
                ORG.append(token.text)
        
        # Appending the relative frequencies per 10000 words and unique NER to the dataframe
        RelFreq_NOUN = round(len(nouns) / (len(file_text) - non_words_counter) * 10000, 2)
        RelFreq_VERB = round(len(verbs) / (len(file_text) - non_words_counter) * 10000, 2)
        RelFreq_ADJ = round(len(adjectives) / (len(file_text) - non_words_counter) * 10000, 2)
        RelFreq_ADV = round(len(adverbs) / (len(file_text) - non_words_counter) * 10000, 2)
        Unique_PER = len(set(PER))
        Unique_LOC = len(set(LOC))
        Unique_ORG = len(set(ORG))
        
        df = df.append({"text_name": y, "RelFreq_NOUN": RelFreq_NOUN, "RelFreq_VERB": RelFreq_VERB, "RelFreq_ADJ": RelFreq_ADJ, "RelFreq_ADV": RelFreq_ADV, "Unique_PER": Unique_PER, "Unique_LOC": Unique_LOC, "Unique_ORG": Unique_ORG}, ignore_index=True)
    # count amount of nouns, adj, adv, etc.
    
    df.to_csv(os.path.join(os.getcwd(), "out", x + ".csv"))


C:\Users\Aleksander\AppData\Local\Temp\ipykernel_6108\3464980696.py:59: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({"text_name": y, "RelFreq_NOUN": RelFreq_NOUN, "RelFreq_VERB": RelFreq_VERB, "RelFreq_ADJ": RelFreq_ADJ, "RelFreq_ADV": RelFreq_ADV, "Unique_PER": Unique_PER, "Unique_LOC": Unique_LOC, "Unique_ORG": Unique_ORG}, ignore_index=True)
C:\Users\Aleksander\AppData\Local\Temp\ipykernel_6108\3464980696.py:59: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({"text_name": y, "RelFreq_NOUN": RelFreq_NOUN, "RelFreq_VERB": RelFreq_VERB, "RelFreq_ADJ": RelFreq_ADJ, "RelFreq_ADV": RelFreq_ADV, "Unique_PER": Unique_PER, "Unique_LOC": Unique_LOC, "Unique_ORG": Unique_ORG}, ignore_index=True)
C:\Users\Aleksander\AppData\Local\Temp\ipykernel_6108\3464980696.py:59: FutureWarning: The fra